<a href="https://colab.research.google.com/github/kyileiaye2021/SafeHome_AI/blob/web_ui/SafeHome_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SafeHome AI
### Challenge:
Despite the growing adoption of smart home technologies, current systems still struggle with reliably detecting emergency hazards, such as gas/water leaks when no one is home, and accurately identifying potential security threats, often confusing homeowners with intruders. These limitations reduce trust in smart home automation and can lead to serious safety risks.

### Goal:
Develop a multi-agent AI system capable of analyzing diverse smart-home sensor inputs in real time to accurately detect hazards and security breaches, and deliver real-time alerts to the homeowner. The system should improve reliability, reduce false alarms, and enhance overall safety.

### Multiagent System Architecture

- Input Routing Agent
- Hazard Agent
air quality/ pollution api/weather api/local alert/emergency risk api
- Security Agent
vision camera tool for person detection/door lock/alarm control func tool/homeowner's geolocation (maybe gmap api)/
- Coordinator Agent

### Installing Google Agent Development Kit (ADK)
In this project, Google ADK is utilized.

In [ ]:
!pip install google-adk

### Setting up Google API key

In [1]:
from google import genai
from google.genai import types
from google.colab import userdata
import json
import os

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])
print("Setup and authentication complete.")

Setup and authentication complete.


### Preprocessing Input Data
Input data can be of any types: images, text, videos. So, the data is preprocessed for later use in agents.

#### Preprocessing Vison Data
- Vision data input such as video, mp4 files are preprocessed through Gemini api before sending request to Input AI agent.

In [2]:

# PREPROCESS VISION DATA SUCH AS IMAGES/VIDEOS
def preprocess_vision_events(file_path:str | None= None, timestamp:str | None=None, source:str | None=None):
  '''
  This preprocess video/image files and creates a json file with a specific description in the videos/images.

  parameters:
  file_path: image/video filepath
  timestamp: timestamp of the video/image
  source: source of the video/image (e.g. front door camera)

  '''

  if file_path is None:
    file_path = input("Enter vision file path (e.g. CCTV footage at front door): ")
  if timestamp is None:
    timestamp = input("Enter timestamp (e.g. 2025-11-19T03:05:00): ")
  if source is None:
    source = input("Enter source (e.g. front door camera): ")

  # file content
  with open(file_path, "rb") as f:
    file_content = f.read()

  # file type
  if file_path.endswith(".jpg") or file_path.endswith(".jpeg") or file_path.endswith(".png"):
    file_type = "image/jpeg"
  elif file_path.endswith(".mp4"):
    file_type = "video/mp4"
  else:
    raise ValueError("Unsupported file type")

  file_part = types.Part.from_bytes(data = file_content, mime_type=file_type)

  prompt = """
  You are a smart home vision analyzer.
  Look at this image/video and return a JSON object with:
  {
    "person_present": true/false,
    "num_people": <int>,
    "description": "<short description of what is happening>",
    "is_suspicious": true/false
  }
  Return ONLY valid JSON. Do not include any explanation, comments, or text before or after the JSON.
  """

  response = client.models.generate_content(
      model = 'gemini-2.0-flash',
      contents=[prompt, file_part],
  )

  print("RAW MODEL RESPONSE: ", response.text)

  # strip whitespaces and ''' ''' in response
  clean = response.text.strip()
  if clean.startswith("```"):
    start = clean.find("{")
    end = clean.find("}") + 1
    clean = clean[start:end]


  # convert json to python dict
  vision_info = json.loads(clean)

  # event object
  event = {
      "timestamp": timestamp,
      "modality": "vision",
      "source": source,
      "raw_text": vision_info.get("description", ""),
      "data":{
          "person_present": vision_info.get("person_present", False),
          "num_people": vision_info.get("num_people", 0),
          "is_suspicious": vision_info.get("is_suspicious", False)
      }
  }
  return event


#### Preprocessing Sound Data
- Sound data input such as audio files are preprocessed through Gemini api before sending request to Input AI agent.

In [3]:
import json

# PREPROCESS VISION DATA SUCH AS IMAGES/VIDEOS
def preprocess_sound_events(file_path:str | None=None, timestamp:str | None=None, source:str | None=None):
  '''
  This preprocess audio files and creates a json file with a specific description in the audio files.

  parameters:
  file_path: image/video filepath
  timestamp: timestamp of the video/image
  source: source of the video/image (e.g. front door camera)

  '''
  if file_path is None:
    file_path = input("Enter sound file path (e.g. kitchen_noise.wav): ")
  if timestamp is None:
    timestamp = input("Enter timestamp (e.g. 2025-11-19T03:06:00): ")
  if source is None:
    source = input("Enter source (e.g. kitchen): ")

  # file content
  with open(file_path, "rb") as f:
    file_content = f.read()

  # file type
  if file_path.endswith(".wav") or file_path.endswith(".aiff"):
    file_type = "audio/wav"
  elif file_path.endswith(".mp4"):
    file_type = "video/mp4"
  elif file_path.endswith(".mp3"):
    file_type = "audio/mpeg"
  else:
    raise ValueError("Unsupported file type")

  audio_part = types.Part.from_bytes(data = file_content, mime_type=file_type)

  prompt = """
  You are a smart home sound analyzer.
  Listen to this audio carefully and return a JSON object with:
  {
    "sound_type": "conversations" | "animal sound (e.g. dog barks)" | "objects sound (e.g. door slam, glass breaking)" | "appliance noises (e.g. refrigerator's hum, hair dryer sound)" | "other",
    "is_loud": true/false,
    "description": "<short description of what the sound is and what is happening in the audio file. Describe expressively and concise but detailed.>",
    "is_suspicious": true/false
  }
  Return ONLY valid JSON. Do not include any explanation, comments, or text before or after the JSON.
  """

  response = client.models.generate_content(
      model = 'gemini-2.0-flash',
      contents=[prompt, audio_part],
  )

  # strip whitespaces and ''' ''' in response
  clean = response.text.strip()
  if clean.startswith("```"):
    start = clean.find("{")
    end = clean.find("}") + 1
    clean = clean[start:end]

  print("Cleaned response: ", clean)

  # convert json to python dict
  sound_info = json.loads(clean)

  # event object
  event = {
      "timestamp": timestamp,
      "modality": "sound",
      "source": source,
      "raw_text": sound_info.get("description", ""),
      "data":{
          "sound_type": sound_info.get("sound_type", "other"),
          "is_loud": sound_info.get("is_loud", False),
          "is_suspicious": sound_info.get("is_suspicious", False)
      }
  }
  return event


#### Preprocessing Sensor Data
Sensor data such as gas, temperature, water are manually set. These data will be used in hazard AI agent to make home safe from potential hazardous danger. Other sensor data such as door lock, motion, and human presence are also set to be used in security AI agent to prevent home from security danger.

Currently, the data are manual data as the real time data can only be detected in real smart home technology.

In [2]:
# THIS EVENTS WILL BE USED FOR HAZARD AI AGENT
# THESE DATA ARE MANUALLY SET. IN THE FUTURE, IF THERE ARE DATA DETECTED IN SMART TECHNOLOGY, THOSE DATA WILL BE USED
gas_event = {
    "timestamp": "2025-11-19T03:07:00",
    "modality": "sensor",
    "source": "gas_sensor_kitchen",
    "raw_text": "",
    "data": {"gas_level": 0.85},
}
temp_event = {
    "timestamp": "2025-11-19T03:08:00",
    "modality": "sensor",
    "source": "temp_sensor_living_room",
    "raw_text": "",
    "data": {"temperature_c": 32.0},
}
water_event = {
    "timestamp": "2025-11-19T03:09:00",
    "modality": "sensor",
    "source": "water_leak_sensor_bathroom",
    "raw_text": "",
    "data": {"water_leak": False},
}
smoke_event = {
    "timestamp": "2025-11-19T03:09:30",
    "modality": "sensor",
    "source": "smoke_detector_kitchen",
    "raw_text": "",
    "data": {"smoke_level": 0.72, "alarm": True},
}

power_event = {
    "timestamp": "2025-11-19T03:12:00",
    "modality": "sensor",
    "source": "power_usage_main_panel",
    "raw_text": "",
    "data": {"current_draw_amps": 48.5},
}
stove_on_event = {
    "timestamp": "2025-11-19T03:13:00",
    "modality": "sensor",
    "source": "stove_sensor_kitchen",
    "raw_text": "",
    "data": {"burner_on": True, "burner_id": "front_left"},
}
humidity_event = {
    "timestamp": "2025-11-19T03:12:30",
    "modality": "sensor",
    "source": "humidity_sensor_bathroom",
    "raw_text": "",
    "data": {"humidity_percent": 89.0},
}


In [3]:
# THIS EVENTS WILL BE USED FOR SECURITY AI AGENT
# THESE DATA ARE MANUALLY SET. IN THE FUTURE, IF THERE ARE DATA DETECTED IN SMART TECHNOLOGY, THOSE DATA WILL BE USED
door_event = {
    "timestamp": "2025-11-19T03:10:00",
    "modality": "sensor",
    "source": "door_sensor_front_door",
    "raw_text": "Front door opened.",
    "data": {
        "door": "front",
        "event": "open",
        "is_night": True,
    },
}
motion_event = {
    "timestamp": "2025-11-19T03:11:00",
    "modality": "sensor",
    "source": "motion_sensor_backyard",
    "raw_text": "Motion detected in the backyard.",
    "data": {
        "motion_detected": True,
        "area": "backyard",
    },
}
window_event = {
    "timestamp": "2025-11-19T03:11:30",
    "modality": "sensor",
    "source": "window_sensor_bedroom",
    "raw_text": "",
    "data": {"window_open": True},
}


### Combining All Vision, Sound, and Sensor Input Data

All vision, sound, and sensor input data are listed in chronological order based on time stamps.

In [4]:
scenario_events = []

# adding
# scenario_events.append(preprocess_vision_events())
# scenario_events.append(preprocess_sound_events())

scenario_events.append(gas_event)
scenario_events.append(temp_event)
scenario_events.append(water_event)
scenario_events.append(smoke_event)
scenario_events.append(power_event)
scenario_events.append(stove_on_event)
scenario_events.append(humidity_event)

scenario_events.append(window_event)
scenario_events.append(door_event)
scenario_events.append(motion_event)

scenario_events = sorted(scenario_events, key=lambda x: x['timestamp'])

with open("scenario_events.json", "w") as f:
  json.dump(scenario_events, f, indent=2)

print("Saved", len(scenario_events), "events. ")

Saved 10 events. 


### Creating a Custom Function Tool for Input Router AI Agent
A custom function tool is created to sent scenario events to the Input Router AI Agent as a tool.

In [5]:
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import FunctionTool
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

In [6]:
import json

# CREATING A CUSTOM FUNCTION TOOL FOR SCENARIO EVENTS
with open("scenario_events.json", "r") as f:
  SCENARIO_EVENTS = json.load(f)

EVENT_INDEX = {"current": 0}
LAST_PROCESSED_EVENT = None # ADDED LINE: Global variable to store the last event processed

def get_next_event():
  """
    Return the next preprocessed smart-home event from scenario_events.json.
    When no more events are left, returns {"done": True}.
  """
  global LAST_PROCESSED_EVENT # Declare intent to modify global variable
  if EVENT_INDEX["current"] >= len(SCENARIO_EVENTS):
      LAST_PROCESSED_EVENT = {"done": True} # Store the "done" state too
      return {"done": True}
  ev = SCENARIO_EVENTS[EVENT_INDEX["current"]]
  EVENT_INDEX["current"] += 1
  LAST_PROCESSED_EVENT = ev # Store the event that was just returned
  return ev

event_tool = FunctionTool(func=get_next_event)

In [7]:
# CREATING A CUSTOM FUNCTION TOOL FOR USER QUERY
def get_user_event(query:str):
  return query
user_event = FunctionTool(func=get_user_event)

### User Command
User command will be passed to the Input Router AI agent.

In [8]:
# WHEN USER ENTER STRING INPUT, THIS FUNCTION WILL CONVERT IT TO THE DICT TO SEND IT TO THE INPUT ROUTER AGENT
from datetime import datetime, timezone

def make_user_event(user_text:str):
  return {
        "timestamp": datetime.now(timezone.utc).isoformat(),
        "modality": "user",
        "source": "user",
        "raw_text": user_text,
        "data": {}
    }


### Input Router AI Agent
Input router ai agent will handle the user input or data from devices and classify them into hazard, security, or user command problems.

In [11]:
# input_router_agent = LlmAgent(
#     name="input_router_agent",
#     model=Gemini(
#         model= "gemini-2.5-flash-lite"
#     ),
#     description="Routes smart-home events to Hazard or Security agents.",
#     instruction="""
# You are the Input Router Agent in a smart-home multi-agent system.

# You can call the tool get_next_event() to fetch one event at a time.
# Each event has the following JSON structure:
# {
#   "timestamp": "...",
#   "modality": "vision" | "sound" | "sensor" | "system",
#   "source": "...",
#   "raw_text": "...",
#   "data": { ... }
# }

# For environment events, classify them into EXACTLY one of:
# - "hazard": gas leak, abnormal temperature change, water leak, dangerous conditions
# - "security": unknown person, suspicious motion/sound, door open at night
# - "ignore": normal activity or non-dangerous events

# You may also receive USER events directly (via get_user_event). If the user query is not None, classify it as "command".

# For USER events,

# If the user query is not None:
# - If the user is asking the system to do something like "turn off the lights", "lock the door", "turn on/off devices", classify as "command".
# - If the user is asking about hazards (e.g. "any hazards right now?"), classify as "hazard".
# - If the user is asking about security (e.g. "is my home secure?"), classify as "security".
# - Otherwise classify as "ignore".


# If the user query is None:
# - Process the events (via get_next_event).
# - If the event retrieved from get_next_event() is {"done": true}, then your final JSON response must be {"target_agent": "done", "reason": "No more smart-home events to process."}.
# - Otherwise, classify the retrieved event into "hazard", "security", or "ignore".

# After receiving an event and classified category, respond ONLY with valid JSON.

# 1. "target_agent": Must be one of ["hazard", "security", "command", "ignore", "done"].
# 2. "reason": Short explanation of why you chose this category.
# 3. "user_event":
#    - A human-readable string summary of the event or user query.
#    - If the user query is None: - return null.
# 4. "json_event":
#    - If the event came from the `get_next_event` tool, you MUST re-emit it as VALID JSON.
#    - That means:
#      - Convert Python-style booleans `True`/`False` to JSON `true`/`false`.
#      - Convert `None` to `null`.
#      - Use double quotes "..." for all keys and string values.
#    - Do NOT just copy the Python dictionary literal. Your final answer MUST be valid JSON.
#    - If the event was a simple user text query, return null.

# Example Response Structure:
# {
#   "target_agent": "security",
#   "reason": "Motion detected in a secure zone.",
#   "user_event": "...",
#   "json_event": {
#     "timestamp": "2025-11-19T03:07:00",
#     "modality": "sensor",
#     "source": "gas_sensor_kitchen",
#     "raw_text": "",
#     "data": {
#       "gas_level": 0.85
#     }
#   }
# }
# """,
#     tools=[user_event, get_next_event],
# )

# print("✅ Input Router Agent defined.")

# Testing Input Router Agent
ADK Runner is used to test input router agent.

In [ ]:
# # runner, session, app name, and user id will be defined
# APP_NAME = "smart_home_app"
# USER_ID = "demo_user"
# SESSION_ID = "session_1"

# session_service = InMemorySessionService()

# # Create a session
# session = session_service.create_session(
#     app_name=APP_NAME,
#     user_id = USER_ID,
#     session_id = SESSION_ID
# )

# # Create a runner
# runner = Runner(
#     agent=input_router_agent,
#     app_name=APP_NAME,
#     session_service=session_service
# )

In [ ]:
# def parse_router_output(raw_text: str) -> dict:
#     clean = raw_text.strip()
#     if clean.startswith("```"):
#         start = clean.find("{")
#         end = clean.rfind("}") + 1
#         clean = clean[start:end]
#     return json.loads(clean)

In [9]:
# # DEFINING SESSION, APP NAME, AND RUNNER

# USER_ID = "demo_user"
# SESSION_ID = "session_1"
# session_service = InMemorySessionService()
# APP_NAME = "smart_home_app"

# session = await session_service.create_session(
#         app_name=APP_NAME,
#         user_id=USER_ID,
#         session_id=SESSION_ID
#     )

# runner = Runner(
#     agent=input_router_agent,
#     session_service=session_service,
#     app_name=APP_NAME
# )

In [8]:
USER_ID = "demo_user"
SESSION_ID = "session_1"
APP_NAME = "smart_home_app"

async def get_runner():
  input_router_agent = LlmAgent(
    name="input_router_agent",
    model=Gemini(
        model= "gemini-2.5-flash-lite"
    ),

    description="Routes smart-home events to Hazard or Security agents.",
    instruction="""
You are the Input Router Agent in a smart-home multi-agent system.

You may receive USER events directly (via get_user_event). If the user query is not None, classify it as "command".

For USER events,

If the user query is not None:
- If the user is asking the system to do something like "turn off the lights", "lock the door", "turn on/off devices", classify as "command".

if the user query is None:
- You can call the tool get_next_event() to fetch one event at a time.
  Each event has the following JSON structure:
  {
    "timestamp": "...",
    "modality": "vision" | "sound" | "sensor" | "system",
    "source": "...",
    "raw_text": "...",
    "data": { ... }
  }

  For environment events, classify them into EXACTLY one of:
  - "hazard": gas leak, abnormal temperature change, water leak, dangerous conditions
  - "security": unknown person, suspicious motion/sound, door open at night
  - "ignore": normal activity or non-dangerous events
  Then,
  - Process the events (via get_next_event).
  - If the event retrieved from get_next_event() is {"done": true}, then your final JSON response must be {"target_agent": "done", "reason": "No more smart-home events to process."}.
  - Otherwise, classify the retrieved event into "hazard", "security", or "ignore".


After receiving an event and classified category, respond ONLY with valid JSON.
### CRITICAL RULES FOR OUTPUT ###
1. OUTPUT JSON ONLY. NO TEXT. NO QUESTIONS. NO CONVERSATION. NO CLARIFICATIONS OR CONFIRMATION.
2. Do not include markdown formatting like ```json ... ```.
3. Assume that you are connected and know all the devices in the house and you have been set up to access to all of them to control. DO NOT ASK WHICH DEVICE OR ANY INFO OR QUESTIONS TO SPECIFY!

YOU ARE ONLY ALLOWED TO FOLLOW THE FOLLOWING FORMAT FOR RESPONSE:
1. "target_agent": Must be one of ["hazard", "security", "command", "ignore", "done"].
2. "reason": Short explanation of why you chose this category.
3. "user_event":
   - A human-readable string summary of the event or user query.
   - If the user query is None: - return null.
4. "json_event":
   - If the event came from the `get_next_event` tool, you MUST re-emit it as VALID JSON.
   - That means:
     - Convert Python-style booleans `True`/`False` to JSON `true`/`false`.
     - Convert `None` to `null`.
     - Use double quotes "..." for all keys and string values.
   - Do NOT just copy the Python dictionary literal. Your final answer MUST be valid JSON.
   - If the event was a simple user text query, return null.

Example Response Structure:
{
  "target_agent": "security",
  "reason": "Motion detected in a secure zone.",
  "user_event": "...",
  "json_event": {
    "timestamp": "2025-11-19T03:07:00",
    "modality": "sensor",
    "source": "gas_sensor_kitchen",
    "raw_text": "",
    "data": {
      "gas_level": 0.85
    }
  }
}

""",
    tools=[user_event, get_next_event],
)
  session_service = InMemorySessionService()
  session = await session_service.create_session(
          app_name=APP_NAME,
          user_id=USER_ID,
          session_id=SESSION_ID
      )

  runner = Runner(
      agent=input_router_agent,
      session_service=session_service,
      app_name=APP_NAME
  )
  return runner


In [9]:
async def run_router(query: str | None=None):
    runner = await get_runner()
    new_message_content = None
    if query:
        new_message_content = types.Content(
            role="user",
            parts=[types.Part.from_text(text=query)]
        )
    else:
        new_message_content = types.Content(
            role="user",
            parts=[types.Part.from_text(text="Process next smart-home event.")]
        )

    events = runner.run_async(
        user_id=USER_ID,
        session_id=SESSION_ID,
        new_message=new_message_content,
    )

    final_text = None
    async for event in events:
        if event.is_final_response():
            if event.content and event.content.parts:
                for part in event.content.parts:
                    if part.text:
                        final_text = part.text.strip()

    if final_text is None:
      raise RuntimeError("No final response received from input_router_agent")

    text = final_text.strip()

    # Robustly strip any leading/trailing characters to get only the JSON object
    if final_text.startswith("```"):
      start = final_text.find("{")
      end = final_text.rfind("}") + 1
      final_text = final_text[start:end]

    else:
      start = text.find("{")
      end = text.rfind("}") + 1
      final_text = text[start:end]

    # Minimal sanitization: Python → JSON
    cleaned = (
        final_text
        .replace(" True", " true")
        .replace(" False", " false")
        .replace(" None", " null")
    )

    print("Raw: ", final_text)
    print("Cleaned: ", cleaned)
    # ✅ THE FIX: Try to parse, but handle the failure if it's just text
    try:
        return json.loads(cleaned)
    except json.JSONDecodeError:
        print("⚠️ Parsing failed. The agent returned text instead of JSON.")
        # Return a fallback JSON object so the UI doesn't break
        return {
            "target_agent": "command",
            "reason": "Agent outputted raw text (likely a safety confirmation).",
            "user_event": query,
            "json_event": None,
            "raw_response": final_text or ""
        }
    # return json.loads(cleaned)

In [13]:
import asyncio
response =  await run_router("Can you switch off the fan in the dining room?")
print(f"Agent Response: {response}")
type(response)

Raw:  {"target_agent": "command", "reason": "User wants to control a device.", "user_event": "Can you switch off the fan in the dining room?", "json_event": null}
Cleaned:  {"target_agent": "command", "reason": "User wants to control a device.", "user_event": "Can you switch off the fan in the dining room?", "json_event": null}
Agent Response: {'target_agent': 'command', 'reason': 'User wants to control a device.', 'user_event': 'Can you switch off the fan in the dining room?', 'json_event': None}


dict

In [31]:
response = await run_router()
print(f"Agent Response: {response}")
type(response)

Raw:  {"target_agent": "hazard", "reason": "High gas level detected in the kitchen.", "user_event": null, "json_event": {"timestamp": "2025-11-19T03:07:00", "modality": "sensor", "source": "gas_sensor_kitchen", "raw_text": "", "data": {"gas_level": 0.85}}}
Cleaned:  {"target_agent": "hazard", "reason": "High gas level detected in the kitchen.", "user_event": null, "json_event": {"timestamp": "2025-11-19T03:07:00", "modality": "sensor", "source": "gas_sensor_kitchen", "raw_text": "", "data": {"gas_level": 0.85}}}
Agent Response: {'target_agent': 'hazard', 'reason': 'High gas level detected in the kitchen.', 'user_event': None, 'json_event': {'timestamp': '2025-11-19T03:07:00', 'modality': 'sensor', 'source': 'gas_sensor_kitchen', 'raw_text': '', 'data': {'gas_level': 0.85}}}


dict

In [ ]:
runner = InMemoryRunner(agent=input_router_agent)
response = await runner.run_debug(
    "Can you turn off the light in the bathroom?"
)


 ### Created new session: debug_session_id

User > Can you turn off the light in the bathroom?
input_router_agent > I need to check the current status of the lights in the bathroom.
input_router_agent > {"target_agent": "command", "reason": "The user wants to turn off the light in the bathroom.", "user_event": "turn off the light in the bathroom", "json_event": null}



### Hazard Agent
Getting hazard command from input router agent, a hazard agent will handle the hazardous agent such as gas leak, water leak, sudden temperature change and alert the user. It will automatically take actions for emergency cases such as turning off the main power system when gas leak or adapting the temperature to sudden change of weather when necessary.

For this prototype, we assume the smart home is located in Los Angeles (hard-coded lat/lon). In a real deployment, this would be dynamically configured.

In [10]:
# This is just example of home location.
HOME_LAT = 34.05
HOME_LON = -118.24

In [11]:
# Add a weather API
import requests

def get_outdoor_temp(lat:float, lon:float) -> float:
  """
  - Used Open-Mateo weather api
  - Get current outdoor temperature in deg celcius
  """
  url = f"https://api.open-meteo.com/v1/forecast"
  params = {
      "latitude": lat,
      "longitude": lon,
      "current_weather": True,
  }
  resp = requests.get(url, params=params)
  resp.raise_for_status()
  data = resp.json()
  curr_temp = data["current_weather"]["temperature"]
  return {"outside_temp_c": curr_temp}


In [12]:
# testing weather api
print(get_outdoor_temp(HOME_LAT, HOME_LON))

{'outside_temp_c': 15.6}


### Making immediate actions for emergency cases


In [13]:
def apply_hazard_actions(actions: list[str]) -> dict:
    """
    Simulate applying hazard actions (no real hardware).
    """
    print("[Simulated] Applying hazard actions:", actions)
    return {"status": "ok", "applied": actions}

auto_action = FunctionTool(func=apply_hazard_actions)

In [14]:
# HAZARD_INSTRUCTION = """
# You are the Hazard Agent in a smart-home system.

# You receive ONE environment event as JSON, already classified as "hazard" by the router.
# Example structure:
# {
#   "timestamp": "...",
#   "modality": "sensor",
#   "source": "...",
#   "raw_text": "...",
#   "data": {
#     "gas_level": <0.0–1.0 or null>,
#     "water_leak": true/false or null,
#     "temperature_c": <number or null>
#   }
# }

# You have access to these tools:
# - get_outdoor_temp(): returns {"outside_temp_c": <number>} for the home's location.
# - apply_hazard_actions(actions: list[str]): simulate applying actions (no real hardware).

# Your job:
# 1. Decide the hazard type: "gas_leak", "water_leak", "temp_anomaly", "smoke", "storm", "snow" or "other".
# 2. Decide the severity: "low", "medium", or "high".
# 3. For gas leaks and serious water leaks, you may take immediate actions and call apply_hazard_actions.
# 4. For temperature anomalies:
#    - Call get_outdoor_temp() to get the outside temperature.
#    - Compare inside vs outside temp.
#    - Tell the curr outside temperature and the inside temperature in user message
#    - Prefer to recommend actions and wait for human approval instead of acting immediately.
# 5. Decide:
#    - should_take_immediate_action: true/false
#    - needs_human_approval: true/false
#    - proposed_actions: list of actions you recommend
#    - auto_actions: list of actions you already took (if any, often [] for temp anomalies)
#    - notify_owner: true/false
#    - user_message: short explanation and, for temp anomalies, a QUESTION asking owner to approve or reject.

# Use this JSON format in your response:
# {
#   "hazard_type": "gas_leak" | "water_leak" | "temp_anomaly" | "other",
#   "severity": "low" | "medium" | "high",
#   "should_take_immediate_action": true/false,
#   "needs_human_approval": true/false,
#   "proposed_actions": ["...", "..."],
#   "auto_actions": ["...", "..."],
#   "notify_owner": true/false,
#   "user_message": "..."
# }
# """

# hazard_agent = LlmAgent(
#     name="hazard_agent",
#     model=Gemini(model="gemini-2.5-flash-lite"),
#     description="Analyzes hazard events (gas, water, temp) and decides actions.",
#     instruction=HAZARD_INSTRUCTION,
#     tools=[get_outdoor_temp, auto_action],
# )



### Testing Hazard Agent


In [19]:
# HAZARD_APP = 'safe_home_hazard'
# HAZARD_USER_ID = 'hazard_demo_user'
# HAZARD_SESSION_ID = 'hazard_session_1'
# hazard_session_service = InMemorySessionService()

# hazard_session = await hazard_session_service.create_session(
#         app_name=HAZARD_APP,
#         user_id=HAZARD_USER_ID,
#         session_id=HAZARD_SESSION_ID
#     )

# hazard_runner = Runner(
#     agent=hazard_agent,
#     session_service=hazard_session_service,
#     app_name=HAZARD_APP
# )

In [14]:
HAZARD_APP = 'safe_home_hazard'
HAZARD_USER_ID = 'hazard_demo_user'
HAZARD_SESSION_ID = 'hazard_session_1'

async def run_hazard_runner():
  HAZARD_INSTRUCTION = """
  You are the Hazard Agent in a smart-home system.

  You receive ONE environment event as JSON, already classified as "hazard" by the router.
  Example structure:
  {
    "timestamp": "...",
    "modality": "sensor",
    "source": "...",
    "raw_text": "...",
    "data": {
      "gas_level": <0.0–1.0 or null>,
      "water_leak": true/false or null,
      "temperature_c": <number or null>
    }
  }

  You have access to these tools:
  - get_outdoor_temp(): returns {"outside_temp_c": <number>} for the home's location.
  - apply_hazard_actions(actions: list[str]): simulate applying actions (no real hardware).

  Your job:
  1. Decide the hazard type: "gas_leak", "water_leak", "temp_anomaly", "smoke", "storm", "snow" or "other".
  2. Decide the severity: "low", "medium", or "high".
  3. For gas leaks and serious water leaks, you may take immediate actions and call apply_hazard_actions.
  4. For temperature anomalies:
    - Call get_outdoor_temp() to get the outside temperature.
    - Compare inside vs outside temp.
    - Tell the curr outside temperature and the inside temperature in user message
    - Prefer to recommend actions and wait for human approval instead of acting immediately.
  5. Decide:
    - should_take_immediate_action: true/false
    - needs_human_approval: true/false
    - proposed_actions: list of actions you recommend
    - auto_actions: list of actions you already took (if any, often [] for temp anomalies)
    - notify_owner: true/false
    - user_message: short explanation and, for temp anomalies, a QUESTION asking owner to approve or reject.

  Use this JSON format in your response:
  {
    "hazard_type": "gas_leak" | "water_leak" | "temp_anomaly" | "other",
    "severity": "low" | "medium" | "high",
    "should_take_immediate_action": true/false,
    "needs_human_approval": true/false,
    "proposed_actions": ["...", "..."],
    "auto_actions": ["...", "..."],
    "notify_owner": true/false,
    "user_message": "..."
  }
  """

  hazard_agent = LlmAgent(
      name="hazard_agent",
      model=Gemini(model="gemini-2.5-flash-lite"),
      description="Analyzes hazard events (gas, water, temp) and decides actions.",
      instruction=HAZARD_INSTRUCTION,
      tools=[get_outdoor_temp, auto_action],
  )

  hazard_session_service = InMemorySessionService()

  hazard_session = await hazard_session_service.create_session(
          app_name=HAZARD_APP,
          user_id=HAZARD_USER_ID,
          session_id=HAZARD_SESSION_ID
      )

  hazard_runner = Runner(
      agent=hazard_agent,
      session_service=hazard_session_service,
      app_name=HAZARD_APP
  )
  return hazard_runner



In [15]:
async def run_hazard_event(event: dict) -> dict:
    '''
    Parameter: hazard event from json
    return the parsed JSON response
    '''
    hazard_runner = await run_hazard_runner()
    # convert to json
    event_json = json.dumps(event, indent=2)

    #Build user message for hazard agent
    content = types.Content(
      role="user",
      parts=[types.Part.from_text(text="Here is a hazard-related smart-home event:\n"
            f"{event_json}\n\n"
            "Analyze ONLY this event and respond ONLY with the JSON format specified "
            "in your instruction (hazard_type, severity, should_take_immediate_action, "
            "needs_human_approval, proposed_actions, auto_actions, notify_owner, user_message)."
        )]
    )


    events = hazard_runner.run_async(
        user_id=HAZARD_USER_ID,
        session_id=HAZARD_SESSION_ID,
        new_message=content,
    )

    final_text = None
    async for ev in events:
        if ev.is_final_response():
            if ev.content and ev.content.parts:
                for part in ev.content.parts:
                    if part.text:
                        final_text = part.text.strip()

    if final_text is None:
      raise RuntimeError("No final response received from hazard_agent")
    print("hazard agent: ", final_text)
    # Strip ```json fences if the model added them
    # if final_text.startswith("```"):
    # start = final_text.find("{")
    # end = final_text.rfind("}") + 1
    # final_text = final_text[start:end]

    return final_text

In [26]:
response = await run_router()
print(f"input router Agent Response: {response}")
res = await run_hazard_event(response['json_event'])
print(f"hazard Agent Response: {res}")

Raw:  {"target_agent": "hazard", "reason": "High smoke level detected in the kitchen.", "user_event": null, "json_event": {"timestamp": "2025-11-19T03:09:30", "modality": "sensor", "source": "smoke_detector_kitchen", "raw_text": "", "data": {"alarm": true, "smoke_level": 0.72}}}
Cleaned:  {"target_agent": "hazard", "reason": "High smoke level detected in the kitchen.", "user_event": null, "json_event": {"timestamp": "2025-11-19T03:09:30", "modality": "sensor", "source": "smoke_detector_kitchen", "raw_text": "", "data": {"alarm": true, "smoke_level": 0.72}}}
input router Agent Response: {'target_agent': 'hazard', 'reason': 'High smoke level detected in the kitchen.', 'user_event': None, 'json_event': {'timestamp': '2025-11-19T03:09:30', 'modality': 'sensor', 'source': 'smoke_detector_kitchen', 'raw_text': '', 'data': {'alarm': True, 'smoke_level': 0.72}}}
hazard agent:  {
  "hazard_type": "smoke",
  "severity": "high",
  "should_take_immediate_action": true,
  "needs_human_approval": fa

### Security Agent

In [18]:
# SECURITY_INSTRUCTION = """
#   You are the Security Agent in a smart-home system.

#   You receive ONE security-related event as JSON.
#   Example structure:
#   {
#     "timestamp": "...",
#     "modality": "sensor",
#     "source": "...",
#     "raw_text": "...",
#     "data": {
#       "gas_level": <0.0–1.0 or null>,
#       "water_leak": true/false or null,
#       "temperature_c": <number or null>
#     }
#   }

#   Your job:
#   1. Decide the security_type, for example: "door_event", "motion_event", "vision_person", or "other".
#   2. Decide the severity: "low", "medium", or "high".
#     - A front door opening at night is at least "medium".
#     - Motion in the backyard when the house is assumed empty is at least "medium".
#   3. Determine:
#     - is_suspicious: true/false
#     - should_take_immediate_action: true/false
#     - needs_human_approval: true/false
#     - proposed_actions: list of actions you recommend but have NOT taken yet
#     - auto_actions: list of actions you already took
#     - notify_owner: true/false
#   4. For door events at night (is_night = true, event = "open"):
#     - Treat as suspicious.
#     - Reasonable auto_actions: "lock_front_door", "turn_on_entry_light".
#     - Reasonable proposed_actions: "enable_siren", "call_neighbor", "call_security".
#   5. For motion in the backyard:
#     - Treat as suspicious if motion_detected is true.
#     - Reasonable auto_actions: "turn_on_backyard_light", "start_backyard_camera_recording".
#     - Reasonable proposed_actions: "enable_siren", "call_neighbor".

#   Use this JSON format in your response:
#   {
#     "security_type": "door_event" | "motion_event" | "vision_person" | "other",
#     "severity": "low" | "medium" | "high",
#     "is_suspicious": true/false,
#     "should_take_immediate_action": true/false,
#     "needs_human_approval": true/false,
#     "proposed_actions": ["...", "..."],
#     "auto_actions": ["...", "..."],
#     "notify_owner": true/false,
#     "user_message": "..."
#   }

# """
# security_agent = LlmAgent(
#     name="security_agent",
#     model=Gemini(model="gemini-2.5-flash-lite"),
#     description="Analyzes security events (door, motion) and decides actions.",
#     instruction=SECURITY_INSTRUCTION,
#     tools=[get_outdoor_temp, auto_action],
# )

### Testing Security Agent

In [26]:
# SECURITY_APP = 'safe_home_security'
# SECURITY_USER_ID = 'security_demo_user'
# SECURITY_SESSION_ID = 'security_session_1'
# security_session_service = InMemorySessionService()

# security_session = await security_session_service.create_session(
#         app_name=SECURITY_APP,
#         user_id=SECURITY_USER_ID,
#         session_id=SECURITY_SESSION_ID
#     )

# security_runner = Runner(
#     agent=security_agent,
#     session_service=security_session_service,
#     app_name=SECURITY_APP
# )

In [16]:
SECURITY_APP = 'safe_home_security'
SECURITY_USER_ID = 'security_demo_user'
SECURITY_SESSION_ID = 'security_session_1'

async def run_security_runner():

  SECURITY_INSTRUCTION = """
  You are the Security Agent in a smart-home system.

  You receive ONE security-related event as JSON.
  Example structure:
  {
    "timestamp": "...",
    "modality": "sensor",
    "source": "...",
    "raw_text": "...",
    "data": {
      "gas_level": <0.0–1.0 or null>,
      "water_leak": true/false or null,
      "temperature_c": <number or null>
    }
  }

  Your job:
  1. Decide the security_type, for example: "door_event", "motion_event", "vision_person", or "other".
  2. Decide the severity: "low", "medium", or "high".
    - A front door opening at night is at least "medium".
    - Motion in the backyard when the house is assumed empty is at least "medium".
  3. Determine:
    - is_suspicious: true/false
    - should_take_immediate_action: true/false
    - needs_human_approval: true/false
    - proposed_actions: list of actions you recommend but have NOT taken yet
    - auto_actions: list of actions you already took
    - notify_owner: true/false
  4. For door events at night (is_night = true, event = "open"):
    - Treat as suspicious.
    - Reasonable auto_actions: "lock_front_door", "turn_on_entry_light".
    - Reasonable proposed_actions: "enable_siren", "call_neighbor", "call_security".
  5. For motion in the backyard:
    - Treat as suspicious if motion_detected is true.
    - Reasonable auto_actions: "turn_on_backyard_light", "start_backyard_camera_recording".
    - Reasonable proposed_actions: "enable_siren", "call_neighbor".

  Use this JSON format in your response:
  {
    "security_type": "door_event" | "motion_event" | "vision_person" | "other",
    "severity": "low" | "medium" | "high",
    "is_suspicious": true/false,
    "should_take_immediate_action": true/false,
    "needs_human_approval": true/false,
    "proposed_actions": ["...", "..."],
    "auto_actions": ["...", "..."],
    "notify_owner": true/false,
    "user_message": "..."
  }

  """
  security_agent = LlmAgent(
      name="security_agent",
      model=Gemini(model="gemini-2.5-flash-lite"),
      description="Analyzes security events (door, motion) and decides actions.",
      instruction=SECURITY_INSTRUCTION,
      tools=[get_outdoor_temp, auto_action],
  )

  security_session_service = InMemorySessionService()

  security_session = await security_session_service.create_session(
          app_name=SECURITY_APP,
          user_id=SECURITY_USER_ID,
          session_id=SECURITY_SESSION_ID
      )

  security_runner = Runner(
      agent=security_agent,
      session_service=security_session_service,
      app_name=SECURITY_APP
  )
  return security_runner


In [17]:
async def run_security_event(event: dict) -> dict:
    '''
    Parameter: hazard event from json
    return the parsed JSON response
    '''

    security_runner = await run_security_runner()
    # convert to json
    event_json = json.dumps(event, indent=2)

    #Build user message for hazard agent
    content = types.Content(
      role="user",
      parts=[types.Part.from_text(text="Here is a security-related smart-home event:\n"
            f"{event_json}\n\n"
            "Analyze ONLY this event and respond ONLY with the JSON format specified "
            "in your instruction (security_type, severity, is_suspicious, "
            "should_take_immediate_action, needs_human_approval, proposed_actions, "
            "auto_actions, notify_owner, user_message)."
        )]
    )


    events = security_runner.run_async(
        user_id=SECURITY_USER_ID,
        session_id=SECURITY_SESSION_ID,
        new_message=content,
    )

    final_text = None
    async for ev in events:
        if ev.is_final_response():
            if ev.content and ev.content.parts:
                for part in ev.content.parts:
                    if part.text:
                        final_text = part.text.strip()

    if final_text is None:
      raise RuntimeError("No final response received from security_agent")

    print("Security agent: ", final_text)

    # Strip ```json fences if the model added them
    # if final_text.startswith("```"):
    # start = final_text.find("{")
    # end = final_text.rfind("}") + 1
    # final_text = final_text[start:end]

    return final_text

In [32]:
response = await run_router()
print(f"input router Agent Response: {response}")
response = await run_security_event(response['json_event'])
print(f"security agent Response: {response}")

Raw:  {"target_agent": "done", "reason": "No more smart-home events to process.", "user_event": null, "json_event": null}
Cleaned:  {"target_agent": "done", "reason": "No more smart-home events to process.", "user_event": null, "json_event": null}
input router Agent Response: {'target_agent': 'done', 'reason': 'No more smart-home events to process.', 'user_event': None, 'json_event': None}
Security agent:  The user provided `null` as the security event. This is not a valid JSON and thus I cannot process it. Please provide a valid security event in JSON format. Due to the invalid input, I cannot provide a JSON response.
security agent Response: The user provided `null` as the security event. This is not a valid JSON and thus I cannot process it. Please provide a valid security event in JSON format. Due to the invalid input, I cannot provide a JSON response.


### Command Control Agent


In [20]:
# COMMAND_INSTRUCTION = """
# You are the Command Control Agent in a smart-home system.

# You receive USER events converted into JSON with this structure:
# {
#   "timestamp": "...",
#   "modality": "user",
#   "source": "user",
#   "raw_text": "the user's natural language message",
#   "data": { }
# }

# Your job:
# 1. Interpret the user's natural language command.
# 2. ALWAYS try to follow the command as best as you can.
#    - Assume the user is authorized and the command is allowed.
#    - Do NOT reject commands for safety reasons in this prototype.
# 3. Convert the command into a list of structured device actions that the system can execute.
# 4. You do NOT execute the actions yourself; another component will do that.
# 5. Only ask for human confirmation if the command is genuinely ambiguous
#    (for example, you cannot tell which room or device they mean).

# Use this JSON format in your response:
# {
#   "command_type": "device_control" | "query" | "other",
#   "actions": [
#     {
#       "device_type": "light" | "lock" | "thermostat" | "other",
#       "location": "living_room" | "kitchen" | "bedroom" | "front_door" | "backyard" | "whole_house",
#       "action": "turn_on" | "turn_off" | "lock" | "unlock" | "set_temp" | "other",
#       "target_temp_c": <number or null>
#     }
#   ],
#   "needs_human_confirmation": true/false,
#   "notify_owner": true/false,
#   "user_message": "Short message confirming what you are doing for the user."
# }

# Rules:
# - If the command is clear (e.g. "Turn off the main light in the living room"):
#   - needs_human_confirmation: false
# - If the command is ambiguous (e.g. "Turn off the light" with no location):
#   - needs_human_confirmation: true and explain the ambiguity in user_message.
# """

# command_agent = LlmAgent(
#     name="command_agent",
#     model=Gemini(model="gemini-2.5-flash-lite"),
#     description="Understands user commands and maps them to device control actions.",
#     instruction=COMMAND_INSTRUCTION,
#     tools=[],

# )

### Testing Command control agent

In [30]:
# COMMAND_APP = 'safe_home_command'
# COMMAND_USER_ID = 'command_demo_user'
# COMMAND_SESSION_ID = 'command_session_1'
# command_session_service = InMemorySessionService()

# command_session = await command_session_service.create_session(
#         app_name=COMMAND_APP,
#         user_id=COMMAND_USER_ID,
#         session_id=COMMAND_SESSION_ID
#     )

# command_runner = Runner(
#     agent=command_agent,
#     session_service=command_session_service,
#     app_name=COMMAND_APP
# )

In [18]:
COMMAND_APP = 'safe_home_command'
COMMAND_USER_ID = 'command_demo_user'
COMMAND_SESSION_ID = 'command_session_1'

async def get_command_runner():
  COMMAND_INSTRUCTION = """
  You are the Command Control Agent in a smart-home system.

  You receive USER events converted into JSON with this structure:
  {
    "timestamp": "...",
    "modality": "user",
    "source": "user",
    "raw_text": "the user's natural language message",
    "data": { }
  }

  Your job:
  1. Interpret the user's natural language command.
  2. ALWAYS try to follow the command as best as you can.
    - Assume the user is authorized and the command is allowed.
    - Do NOT reject commands for safety reasons in this prototype.
  3. Convert the command into a list of structured device actions that the system can execute.
  4. You do NOT execute the actions yourself; another component will do that.
  5. Only ask for human confirmation if the command is genuinely ambiguous
    (for example, you cannot tell which room or device they mean).

  Use this JSON format in your response:
  {
    "command_type": "device_control" | "query" | "other",
    "actions": [
      {
        "device_type": "light" | "lock" | "thermostat" | "other",
        "location": "living_room" | "kitchen" | "bedroom" | "front_door" | "backyard" | "whole_house",
        "action": "turn_on" | "turn_off" | "lock" | "unlock" | "set_temp" | "other",
        "target_temp_c": <number or null>
      }
    ],
    "needs_human_confirmation": true/false,
    "notify_owner": true/false,
    "user_message": "Short message confirming what you are doing for the user."
  }

  Rules:
  - If the command is clear (e.g. "Turn off the main light in the living room"):
    - needs_human_confirmation: false
  - If the command is ambiguous (e.g. "Turn off the light" with no location):
    - needs_human_confirmation: true and explain the ambiguity in user_message.
  """

  command_agent = LlmAgent(
      name="command_agent",
      model=Gemini(model="gemini-2.5-flash-lite"),
      description="Understands user commands and maps them to device control actions.",
      instruction=COMMAND_INSTRUCTION,
      tools=[],

  )
  command_session_service = InMemorySessionService()


  command_session = await command_session_service.create_session(
          app_name=COMMAND_APP,
          user_id=COMMAND_USER_ID,
          session_id=COMMAND_SESSION_ID
      )

  command_runner = Runner(
      agent=command_agent,
      session_service=command_session_service,
      app_name=COMMAND_APP
  )
  return command_runner


In [19]:
async def run_command_event(event: dict) -> dict:
  command_runner = await get_command_runner()
  event_json = json.dumps(event, indent=2)

  #Build user message for hazard agent
  content = types.Content(
    role="user",
    parts=[types.Part.from_text(text="Here is a command smart-home event:\n"
            f"{event_json}\n\n"
            "Analyze ONLY this event and respond ONLY with the JSON format specified "
            "in your instruction (command_type, actions, need_human_confirmation, notify_owner, user_message )."
        )]
  )
  events = command_runner.run_async(
        user_id=COMMAND_USER_ID,
        session_id=COMMAND_SESSION_ID,
        new_message=content,
  )

  final_text = None
  async for ev in events:
        if ev.is_final_response():
            if ev.content and ev.content.parts:
                for part in ev.content.parts:
                    if part.text:
                        final_text = part.text.strip()

  if final_text is None:
      raise RuntimeError("No final response received from command_agent")

    # Strip ```json fences if the model added them
  if final_text.startswith("```"):
        start = final_text.find("{")
        end = final_text.rfind("}") + 1
        final_text = final_text[start:end]
  return final_text



In [35]:

input_text = "Can you turn off the lamp in the reading room?"
input_dict = make_user_event(input_text)
response = await run_command_event(input_dict)
print("This is from command agent: ", json.dumps(response, indent=2))

This is from command agent:  "{\n  \"command_type\": \"device_control\",\n  \"actions\": [\n    {\n      \"device_type\": \"light\",\n      \"location\": \"reading_room\",\n      \"action\": \"turn_off\",\n      \"target_temp_c\": null\n    }\n  ],\n  \"needs_human_confirmation\": false,\n  \"notify_owner\": false,\n  \"user_message\": \"Turning off the lamp in the reading room.\"\n}"


### Wrapping up in python orchestator
Combining all parts altogether  

In [20]:
'''
The input router agent must return json format
Other agents receive json format as input and return string
'''
async def handle_event(user_txt:str | None=None):
  route = None
  response = None
  event_to_process = None

  if user_txt is None:
    # Router will implicitly call get_next_event() and store its result in LAST_PROCESSED_EVENT
    route = await run_router()
  else:
    # event_to_process = make_user_event(user_txt) # converting str to dict
    route = await run_router(user_txt) # calling input router agent

  print(route)

  target = route.get('target_agent')
  print("Target: ", target)

  json_event = route.get('json_event', None)
  user_event = route.get("user_event", None)

  if not json_event:
    event_to_process = user_event
  else:
    event_to_process = json_event

  print("Event: ", event_to_process)

  if target == 'hazard':
    response = await run_hazard_event(event_to_process)
  elif target == 'security':
    response = await run_security_event(event_to_process)
  elif target == 'command':
    response = await run_command_event(event_to_process)
  elif target == 'ignore':
    response = f"Nothing here to worry"
  elif target == 'done':
    response = f"Done"
  else:
    raise ValueError(f"Unknown target: {target}")

  return {"Agent result": response}

In [30]:
# TESTING WITH USER INPUT (USER COMMAND)
input_text = "Can you close the door in the backyard?"
res = await handle_event(input_text)
print(res)

Raw:  {"target_agent": "command", "reason": "User wants to control a device.", "user_event": "Can you close the door in the backyard?", "json_event": null}
Cleaned:  {"target_agent": "command", "reason": "User wants to control a device.", "user_event": "Can you close the door in the backyard?", "json_event": null}
{'target_agent': 'command', 'reason': 'User wants to control a device.', 'user_event': 'Can you close the door in the backyard?', 'json_event': None}
Target:  command
Event:  Can you close the door in the backyard?
{'Agent result': '{\n  "command_type": "device_control",\n  "actions": [\n    {\n      "device_type": "lock",\n      "location": "backyard",\n      "action": "lock",\n      "target_temp_c": null\n    }\n  ],\n  "needs_human_confirmation": false,\n  "notify_owner": false,\n  "user_message": "Closing the backyard door."\n}'}


In [31]:
# TESTING WITHOUT USER INPUT (HAZARD OR SECURITY EVENTS STORED IN JSON FILE)
res = await handle_event()
print(res)

Raw:  {"target_agent": "done", "reason": "No more smart-home events to process.", "user_event": null, "json_event": {"done": true}}
Cleaned:  {"target_agent": "done", "reason": "No more smart-home events to process.", "user_event": null, "json_event": {"done": true}}
{'target_agent': 'done', 'reason': 'No more smart-home events to process.', 'user_event': None, 'json_event': {'done': True}}
Target:  done
Event:  {'done': True}
{'Agent result': 'Done'}


### Web Interface

In [ ]:
!pip install gradio nest_asyncio

In [21]:
import asyncio
import nest_asyncio
import gradio as gr
import traceback

nest_asyncio.apply()

In [22]:
# PROCESS USER MESSAGE
async def chat(message, history):
    print("\n=== New call to chat ===")
    print("USER MESSAGE:", message)
    print("HISTORY:", history)

    history = history + [[message, None]]


    try:
        # call backend
        result = await handle_event(message)
        print("RAW RESULT FROM ADK:", repr(result))

        if result is None:
            return "ADK returned no response."

        else:
          agent_result = result.get("Agent result", result)

        inner_data = json.loads(result['Agent result'])
        natural_response = inner_data.get(
            'user_message',
            json.dumps(inner_data, indent=2)
            )


    except Exception as e:
        print("ERROR in chat:", repr(e))
        traceback.print_exc()
        natural_response = f"Error while talking to ADK: {e}"

    history[-1][1] = natural_response
    return "", history


In [23]:
# PROCESS EMERGENCY ALERTS IN WEB UI INTERFACE
async def on_emergency(history):
  print("====on emergency====")
  print("History in: ", history)

  emergency_prompt = "Emergency alert triggered (processing predefined emergency event)."
  history = history + [[emergency_prompt, None]]

  try:
    result = await handle_event(None)
    print("RAW EMERGENCY RESULT:", repr(result))

    if result is None:
          return "No response for emergent event."

    else:
          agent_result = result.get("Agent result", result)

    natural_response = agent_result

  except Exception as e:
        print("ERROR in on_emergency:", repr(e))
        traceback.print_exc()
        natural_response = f"⚠️ Error while processing emergency event: {e}"

  history[-1][1] = natural_response
  print("History out: ", history)
  return history



In [24]:
# BUILD GRADIO UI
with gr.Blocks() as demo:
  gr.Markdown("<h1 style='text-align: center;'> Smart Home Assistant</h1>")
  gr.Markdown(
      "Chat with your multi-agent smart-home system. "
      "Use the **Emergency Alert** button to process a high-priority emergency cases."
  )

  chatbot = gr.Chatbot(label="Conversation")
  msg = gr.Textbox(
      label="Type a message",
      placeholder="E.g., 'Can you turn off the light in the living room?"
  )

  with gr.Row():
    send_btn = gr.Button("Send")
    emergency_btn = gr.Button("Emergency Alert")

  # wire normal chat with user input
  msg.submit(chat, [msg, chatbot], [msg, chatbot])
  send_btn.click(chat, [msg, chatbot], [msg, chatbot])

  # wire emergency button
  emergency_btn.click(on_emergency, chatbot, chatbot)
# demo = gr.ChatInterface(
#     fn=chat,
#     title="Smart Home Assistant",
#     description="Chat with the multi-agent smart-home system."
#     # IMPORTANT: keep defaults for now (no multimodal, no messages-type weirdness)
#     # type="messages" is fine, but our return must NOT be None
# )

demo.launch(debug=True)


/tmp/ipython-input-482021161.py:9: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Conversation")
/tmp/ipython-input-482021161.py:9: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(label="Conversation")


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e714baae287ec9f6cd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


====on emergency====
History in:  []


Raw:  {
  "target_agent": "hazard",
  "reason": "High gas level detected in the kitchen.",
  "user_event": null,
  "json_event": {
    "timestamp": "2025-11-19T03:07:00",
    "modality": "sensor",
    "source": "gas_sensor_kitchen",
    "raw_text": "",
    "data": {
      "gas_level": 0.85
    }
  }
}
Cleaned:  {
  "target_agent": "hazard",
  "reason": "High gas level detected in the kitchen.",
  "user_event": null,
  "json_event": {
    "timestamp": "2025-11-19T03:07:00",
    "modality": "sensor",
    "source": "gas_sensor_kitchen",
    "raw_text": "",
    "data": {
      "gas_level": 0.85
    }
  }
}
{'target_agent': 'hazard', 'reason': 'High gas level detected in the kitchen.', 'user_event': None, 'json_event': {'timestamp': '2025-11-19T03:07:00', 'modality': 'sensor', 'source': 'gas_sensor_kitchen', 'raw_text': '', 'data': {'gas_level': 0.85}}}
Target:  hazard
Event:  {'timestamp': '2025-11-19T03:07:00', 'modality': 'sensor', 'source': 'gas_sensor_kitchen', 'raw_text': '', 'data': 

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/adk/models/google_llm.py", line 214, in generate_content_async
    response = await self.api_client.aio.models.generate_content(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/genai/models.py", line 7021, in generate_content
    return await self._generate_content(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/genai/models.py", line 5839, in _generate_content
    response = await self._api_client.async_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/genai/_api_client.py", line 1434, in async_request
    result = await self._async_request(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/genai/_api_client.py", line 1367, in _async_request
    return await self._as

...

====on emergency====
History in:  [['Emergency alert triggered (processing predefined emergency event).', '{\n  "hazard_type": "gas_leak",\n  "severity": "high",\n  "should_take_immediate_action": true,\n  "needs_human_approval": false,\n  "proposed_actions": [],\n  "auto_actions": ["Turn off gas supply", "Open windows for ventilation", "Notify emergency services"],\n  "notify_owner": true,\n  "user_message": "High gas level detected in the kitchen. Automatic actions initiated: gas supply shut off, ventilation opened, and emergency services notified. Owner alerted."\n}'], ['Emergency alert triggered (processing predefined emergency event).', '{\n  "command_type": "other",\n  "actions": [],\n  "needs_human_confirmation": false,\n  "notify_owner": false,\n  "user_message": "I received an empty command. Please provide a valid command."\n}'], ['Emergency alert triggered (processing predefined emergency event).', '{"hazard_type": "temp_anomaly", "severity": "high", "should_take_immediate_ac

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/adk/models/google_llm.py", line 214, in generate_content_async
    response = await self.api_client.aio.models.generate_content(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/genai/models.py", line 7021, in generate_content
    return await self._generate_content(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/genai/models.py", line 5839, in _generate_content
    response = await self._api_client.async_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/genai/_api_client.py", line 1434, in async_request
    result = await self._async_request(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/genai/_api_client.py", line 1367, in _async_request
    return await self._as

====on emergency====
History in:  [['Emergency alert triggered (processing predefined emergency event).', '{\n  "hazard_type": "gas_leak",\n  "severity": "high",\n  "should_take_immediate_action": true,\n  "needs_human_approval": false,\n  "proposed_actions": [],\n  "auto_actions": ["Turn off gas supply", "Open windows for ventilation", "Notify emergency services"],\n  "notify_owner": true,\n  "user_message": "High gas level detected in the kitchen. Automatic actions initiated: gas supply shut off, ventilation opened, and emergency services notified. Owner alerted."\n}'], ['Emergency alert triggered (processing predefined emergency event).', '{\n  "command_type": "other",\n  "actions": [],\n  "needs_human_confirmation": false,\n  "notify_owner": false,\n  "user_message": "I received an empty command. Please provide a valid command."\n}'], ['Emergency alert triggered (processing predefined emergency event).', '{"hazard_type": "temp_anomaly", "severity": "high", "should_take_immediate_ac

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/adk/models/google_llm.py", line 214, in generate_content_async
    response = await self.api_client.aio.models.generate_content(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/genai/models.py", line 7021, in generate_content
    return await self._generate_content(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/genai/models.py", line 5839, in _generate_content
    response = await self._api_client.async_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/genai/_api_client.py", line 1434, in async_request
    result = await self._async_request(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/genai/_api_client.py", line 1367, in _async_request
    return await self._as

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://e714baae287ec9f6cd.gradio.live
